# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('tic-tac-toe.csv')

# inspect 
print(data.head())
print(data.info())
#my eyeballs say it looks ok

# categorical values to numeric
data = data.replace({'x': 1, 'o': 0, 'b': -1})

# inputs and output
X = data.drop('class', axis=1)
y = data['class'].astype(int)

# normalize X=input data
scaler = StandardScaler()
X = scaler.fit_transform(X)


  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

# Check for GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is available")
    except RuntimeError as e:
        print(e)
else:
    print("GPU is not available")
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create the model
model = Sequential([
    Dense(128, input_shape=(9,), activation='relu'), # extra layer
    Dropout(0.2),  # adding dropout for regularization
    Dense(64, activation='relu'),
    Dropout(0.2),  # adding dropout for regularization
    Dense(32, activation='relu'), 
    Dense(2, activation='softmax')
])

# Adjust the learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)  # lower learning rate

# compile
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# fit
history = model.fit(X_train, y_train, epochs=150, validation_split=0.2, batch_size= 32)

# evaluate
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

# save
model.save('tic-tac-toe.keras')

#Test accuracy: 0.9270833134651184

GPU is not available
Epoch 1/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6539 - loss: 0.6457 - val_accuracy: 0.6818 - val_loss: 0.6060
Epoch 2/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6673 - loss: 0.6204 - val_accuracy: 0.7143 - val_loss: 0.6018
Epoch 3/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6626 - loss: 0.6207 - val_accuracy: 0.7208 - val_loss: 0.5971
Epoch 4/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6735 - loss: 0.5978 - val_accuracy: 0.7078 - val_loss: 0.5947
Epoch 5/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7110 - loss: 0.5676 - val_accuracy: 0.7273 - val_loss: 0.5921
Epoch 6/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7158 - loss: 0.5651 - val_accuracy: 0.7208 - val_loss: 0.5877
Epoch 7/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7134 - loss: 0.5427 - val_accuracy: 0.7338 - val_loss: 0.5769
Epoch 8/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7363 - loss: 0.5331 - va

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [24]:
import numpy as np

loaded_model = tf.keras.models.load_model('tic-tac-toe.keras')

# predictions on random rows
random_indices = np.random.choice(X_test.shape[0], 5, replace=False)
X_random = X_test[random_indices]
y_random = y_test.iloc[random_indices]

predictions = loaded_model.predict(X_random)
predicted_classes = np.argmax(predictions, axis=1)

# Check if the predictions are correct
print("Actual classes: ", y_random.values)
print("Predicted classes: ", predicted_classes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Actual classes:  [1 1 0 1 1]
Predicted classes:  [1 1 0 1 1]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [25]:
# Create the model with additional layers
model = Sequential([
    Dense(128, input_shape=(9,), activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

# custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# more epochs
history = model.fit(X_train, y_train, epochs=150, validation_split=0.2, batch_size=32)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

# 
model.save('tic_tac_toe_model_v1.keras')

Epoch 1/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5602 - loss: 0.6861 - val_accuracy: 0.6818 - val_loss: 0.6487
Epoch 2/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6285 - loss: 0.6513 - val_accuracy: 0.6818 - val_loss: 0.6177
Epoch 3/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6325 - loss: 0.6428 - val_accuracy: 0.6818 - val_loss: 0.6054
Epoch 4/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6511 - loss: 0.6278 - val_accuracy: 0.6818 - val_loss: 0.6048
Epoch 5/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6356 - loss: 0.6427 - val_accuracy: 0.6818 - val_loss: 0.6074
Epoch 6/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6309 - loss: 0.6442 - val_accuracy: 0.6818 - val_loss: 0.6007
Epoch 7/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6670 - loss: 0.5954 - val_accuracy: 0.6818 - val_loss: 0.5930
Epoch 8/150
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6282 - loss: 0.6218 - val_accuracy: 0.6818 - 

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# increasing amount of layers and epochs
#lower learing rate
#Test accuracy: 0.9635416865348816